# Neo4j - Initial Setup

In [7]:
from NeoDB.neo4j_repo import Neo4jRepository
from NeoDB.connection import get_neo4j_driver
import pandas as pd

Connect to Neo4j

In [2]:
driver = get_neo4j_driver()

In [3]:
neo_repo = Neo4jRepository(driver)

Create uniqueness constraint on User.username

In [4]:
neo_repo.run_cypher("CREATE CONSTRAINT IF NOT EXISTS FOR (u:User) REQUIRE u.username IS UNIQUE")

[]

In [5]:
neo_repo.run_cypher("UNWIND $users AS u MERGE (:User {username: u})", {"users": ["alice", "bob", "carol"]})

[]

In [6]:
neo_repo.run_cypher("MATCH (a:User {username:'alice'}), (b:User {username:'bob'}) MERGE (a)-[:FOLLOWS]->(b)")
neo_repo.run_cypher("MATCH (b:User {username:'bob'}), (c:User {username:'carol'}) MERGE (b)-[:FOLLOWS]->(c)")
neo_repo.run_cypher("MATCH (a:User {username:'alice'}), (c:User {username:'carol'}) MERGE (a)-[:FOLLOWS]->(c)")

[]

In [8]:
rows = neo_repo.run_cypher("MATCH (u:User)-[:FOLLOWS]->(v:User) RETURN u.username AS follower, v.username AS followed ORDER BY follower, followed")
pd.DataFrame([dict(r) for r in rows])

,follower,followed
0,alice,bob
1,alice,carol
2,bob,carol
